<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige bibliotek og pakker

In [2]:
!pip install LAMDA-SSL
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.7 MB/s eta 0:00:00


Importering av nødvendige bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

from sklearn.metrics import mean_squared_error, r2_score

import optuna
from optuna.visualization import plot_optimization_history

Importering av relevant data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# Velger første kolonne med dato og tid som index
mw_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                      header=0,
                      sep=',',
                      index_col=0)

# Formatterer index til riktig format og datatype
mw_data.index = pd.to_datetime(mw_data.index,
                               format='%Y-%m-%d %H:%M:%S')

Mounted at /content/drive


Tilfeldighetsfrø

In [5]:
random_seed = 123

Beskrivelse av datasett

In [6]:
mw_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatFlow     29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT07           29136 non-null  float64
 11  TT08           29136 non-null  float64
 12  PT03           29136 non-null  float64
 13  TT20           29136 non-null  float64
 14  TT09           29136 non-null  float64
 15  TT12           29136 non-null  float64
 16  Mw             460 non-null    float64
dtypes: float64(11),

# Oppdeling av trening og testsett

## Alternativ 1: Fordelt på enzymtype

In [9]:
 def oppdeling_alternativ_1(df, test_size=0.2, random_seed=123):

  # Responsvariabel
  responsvariabel = df.columns[-1]

  # Deler opp markert og umarkert del av datasettet
  data_markert = df.dropna(subset=[responsvariabel])
  data_umarkert = df[df[responsvariabel].isna()]

  # Legger til kategorisk variabel for enzymtype til fordeling av datsettet
  data_markert['EnzymType'] = data_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')

  # Splitter datasett i dataasett og kategorisk enzymtype
  enzymetypes = data_markert['EnzymType']
  data_markert.drop(columns=['EnzymType'], inplace=True)

  # Splitter trening og testdata etter enzymtype
  trening_markert, test_markert, _, _ = train_test_split(data_markert,
                                                          enzymetypes,
                                                          test_size=test_size,
                                                          stratify=enzymetypes,
                                                          random_state=random_seed)

  # Definerer trening og testsett for data
  testdata = test_markert

  felles_indeks = df.index.intersection(testdata.index)
  treningsdata = df.drop(felles_indeks)

  return treningsdata, testdata

# NIR målinger inkludert

## Oppdeling av datasett for trening og testing

### Alternativ 1: Fordelt på Enzymtyper

In [7]:
mw_trening, mw_test = oppdeling_alternativ_1(mw_data)
mw_trening_markert = mw_trening.dropna(subset=["Mw"])
mw_trening_umarkert = mw_trening[mw_trening["Mw"].isna()]

<ipython-input-6-fb7e234f3b43>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert['EnzymType'] = data_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')
<ipython-input-6-fb7e234f3b43>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert.drop(columns=['EnzymType'], inplace=True)


In [14]:
X_trening = mw_trening.iloc[:, :-1]
X_trening_markert = mw_trening_markert.iloc[:, :-1]
X_trening_umarkert = mw_trening_umarkert.iloc[:, :-1]
X_test = mw_test.iloc[:, :-1]

y_trening_markert = mw_trening_markert.iloc[:, -1]
y_test = mw_test.iloc[:, -1]

Skalerer data etter fullstendig treningsdata

In [10]:
skalerer = StandardScaler()
skalerer.fit(X_trening)
X_trening_markert_transformert = skalerer.transform(X_trening_markert)
X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
X_test_transformert = skalerer.transform(X_test)

NameError: name 'X_trening' is not defined

## CoReg

Hyperparamter optimalisering med Optuna

In [16]:
def objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=10)

plot_optimization_history(study)

[I 2024-02-08 14:50:21,902] A new study created in memory with name: no-name-d0a2dde2-bc5e-41a7-8a27-8c6fec6b8a0b
[I 2024-02-08 14:52:52,907] Trial 0 finished with value: 295286.37113945733 and parameters: {'k1': 9, 'k2': 4, 'p1': 6, 'p2': 1}. Best is trial 0 with value: 295286.37113945733.
[I 2024-02-08 14:55:18,198] Trial 1 finished with value: 304659.84635782486 and parameters: {'k1': 3, 'k2': 2, 'p1': 6, 'p2': 6}. Best is trial 0 with value: 295286.37113945733.
[I 2024-02-08 14:57:41,591] Trial 2 finished with value: 307479.7336202926 and parameters: {'k1': 10, 'k2': 10, 'p1': 9, 'p2': 9}. Best is trial 0 with value: 295286.37113945733.
[I 2024-02-08 15:00:08,847] Trial 3 finished with value: 298237.55005890067 and parameters: {'k1': 6, 'k2': 4, 'p1': 7, 'p2': 10}. Best is trial 0 with value: 295286.37113945733.
[I 2024-02-08 15:02:35,208] Trial 4 finished with value: 299950.04278170754 and parameters: {'k1': 6, 'k2': 3, 'p1': 6, 'p2': 2}. Best is trial 0 with value: 295286.3711394

Evauleringer av beste modell

In [17]:
def detailed_objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening_markert, y_trening_prediksjon)
  r2_trening = r2_score(y_trening_markert, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for CoReg: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for CoReg: {'k1': 6, 'k2': 2, 'p1': 7, 'p2': 1}
Test: MSE: 285502.1581385265, og R^2: 0.9484401176513998
Trening: MSE: 69639.58314727066, og R^2: 0.9877787579401235


# NIR målinger ekskludert

In [11]:
mw_data_uten_NIR = mw_data.drop(columns=['NIRfat', 'NIRwater', 'NIRash'])
mw_data_uten_NIR.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatFlow     29136 non-null  float64
 7   TT07           29136 non-null  float64
 8   TT08           29136 non-null  float64
 9   PT03           29136 non-null  float64
 10  TT20           29136 non-null  float64
 11  TT09           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Mw             460 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


## Oppdeling av trening og testsett

### Alternativ 1: Fordelt på Enzymtyper

In [21]:
mw_trening, mw_test = oppdeling_alternativ_1(mw_data_uten_NIR)
mw_trening_markert = mw_trening.dropna(subset=["Mw"])
mw_trening_umarkert = mw_trening[mw_trening["Mw"].isna()]

<ipython-input-9-fb7e234f3b43>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert['EnzymType'] = data_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')
<ipython-input-9-fb7e234f3b43>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert.drop(columns=['EnzymType'], inplace=True)


In [22]:
X_trening = mw_trening.iloc[:, :-1]
X_trening_markert = mw_trening_markert.iloc[:, :-1]
X_trening_umarkert = mw_trening_umarkert.iloc[:, :-1]
X_test = mw_test.iloc[:, :-1]

y_trening_markert = mw_trening_markert.iloc[:, -1]
y_test = mw_test.iloc[:, -1]

Skalerer data etter fullstendig treningsdata

In [23]:
skalerer = StandardScaler()
skalerer.fit(X_trening)
X_trening_markert_transformert = skalerer.transform(X_trening_markert)
X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
X_test_transformert = skalerer.transform(X_test)

## CoReg

Hyperparamter optimalisering med Optuna

In [24]:
def objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=10)

plot_optimization_history(study)

[I 2024-02-08 16:21:17,135] A new study created in memory with name: no-name-af15f0a3-205b-4259-bb58-a8e968e6f62a
[I 2024-02-08 16:22:50,812] Trial 0 finished with value: 618435.3733127204 and parameters: {'k1': 2, 'k2': 7, 'p1': 5, 'p2': 10}. Best is trial 0 with value: 618435.3733127204.
[I 2024-02-08 16:24:24,241] Trial 1 finished with value: 606724.3705235827 and parameters: {'k1': 7, 'k2': 10, 'p1': 9, 'p2': 2}. Best is trial 1 with value: 606724.3705235827.
[I 2024-02-08 16:25:58,581] Trial 2 finished with value: 590822.9067157352 and parameters: {'k1': 10, 'k2': 2, 'p1': 9, 'p2': 1}. Best is trial 2 with value: 590822.9067157352.
[I 2024-02-08 16:27:32,230] Trial 3 finished with value: 592902.0827560227 and parameters: {'k1': 8, 'k2': 4, 'p1': 2, 'p2': 4}. Best is trial 2 with value: 590822.9067157352.
[W 2024-02-08 16:28:09,938] Trial 4 failed with parameters: {'k1': 10, 'k2': 1, 'p1': 3, 'p2': 3} because of the following error: KeyboardInterrupt().
Traceback (most recent call 

KeyboardInterrupt: 

Evauleringer av beste modell

In [ ]:
def detailed_objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening_markert, y_trening_prediksjon)
  r2_trening = r2_score(y_trening_markert, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for CoReg: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))